# TODO

Implement babbage sltr query in python and optimise rescore weights for 'all' query ndcg

In [1]:
baseUrl = "http://localhost:20000/"

import urllib, urllib2
from bs4 import BeautifulSoup
from modules.ONS import utils as search_utils
reload(search_utils)

class ONSScraper(object):
    def __init__(self, baseUrl):
        if (baseUrl.endswith("/")):
            baseUrl = str(baseUrl)[:-1]
        self.baseUrl = baseUrl
        
    def getPage(self, extension):
        targetUrl = self.baseUrl + extension
        page = urllib2.urlopen(targetUrl)
        return BeautifulSoup(page)
        
    def searchResultsList(self, searchTerm, method=search_utils.search, **kwargs):
        page = method(searchTerm, **kwargs)
        divElements = page.select("div.search-results")
        if (len(divElements) == 1):
            ulElements = divElements[0].select("ul.list--neutral")
            if (len(ulElements) == 1):
                liElements = ulElements[0].select("li.search-results__item")
                
                results = []
                for elem in liElements:
                    href = elem.select("a")[0].attrs["href"]
                    results.append(href)
                return results
        return None

onsScraper = ONSScraper(baseUrl)

In [2]:
from pymongo import MongoClient, ASCENDING, DESCENDING
client = MongoClient('localhost', 27017)

db = client.local
collection = db.judgements

def mergeDicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

In [3]:
searchTerms = ["rpi", "gender pay gap", "cpi", "gdp", "inflation", "crime", "unemployment", 
              "population", "immigration", "mental health", "london", "london population", 
              "retail price index", "life expectancy", "obesity", "religion", "migration", 
              "poverty", "social media", "employment"]
models = []
for i in range(9):
    models.append("ons_model_%d" % (i+1))
models.append("all")

def termQuery(term):
    return {"term": term}

def timeQuery(dateTime):
    return {"timeStamp": dateTime}

# Get date of most recent entries
doc = collection.find().sort([("timeStamp", DESCENDING)]).limit(1).next()
timeStamp = doc["timeStamp"]
    
timeStampQuery = timeQuery(timeStamp)
        

In [4]:
import numpy as np

MAX_SCORE = 4.0

def idealJudgement(num):
    i = 0
    incremenet = (1.0 / (float(num) - 1.0)) * num
    
    iJ = np.zeros(num)
    val = len(iJ)
    while (val > 0):
        iJ[i] = (val / float(num)) * MAX_SCORE
        i += 1
        val -= incremenet
        
    return iJ

def idealDiscountedCumulativeGain(num):
    idealGain = idealJudgement(num)
    iDCG = np.zeros(num)
    
    total = 0.0
    for i in range(num):
        total += idealGain[i] / float(i+1)
        iDCG[i] = total
    return iDCG

class Judgements(object):
    def __init__(self, qid, judgements):
        self.qid = qid
        self.judgements = judgements
        
    def dcg(self):
        total = 0.0
        
        dcg = []
        
        for i in range(len(self.judgements)):
            judgement = self.judgements[i]
            total += judgement["judgement"] / float(judgement["rank"])
            dcg.append(total)
            
        return np.array(dcg)
    
    def ndcg(self):
        
        dcg = self.dcg()
        idcg = idealDiscountedCumulativeGain(len(dcg))
        
        ndcg = np.zeros(len(dcg))
        
        for i in range(len(ndcg)):
            ndcg[i] = min(1.0, dcg[i] / idcg[i])
        return ndcg
    
    def __iter__(self):
        return self.judgements.__iter__()
    
    def __getitem__(self, i):
        return self.judgements[i]
    
    def __len__(self):
        return len(self.judgements)
    
    def remove(self, item):
        self.judgements.remove(item)

In [5]:
import copy, jwt
from urllib2 import HTTPError
reload(search_utils)

def getUriFromJwtToken(token):
    jwtDecodedDict = jwt.decode(token, 'secret', algorithms=['HS256'])
    return jwtDecodedDict["uri"]

modelPerformanceDict = {}
for model in models:
    modelPerformanceDict[model] = []
    
def processSearchTerm(searchTerm, modelPerformanceDict):
    print "***********************************************"
    print searchTerm
    print "***********************************************"
    # Load the judgements
    query = mergeDicts(termQuery(searchTerm), timeStampQuery)
    cursor = collection.find(query)
    count = cursor.count()
    
    if (count > 0):
        judgements = cursor.next()
        
        # Get search results for each model
        for model in models:
            # Crawl the website
            modelJudgements = copy.deepcopy(judgements)

            # Scrape new ranks
            judgementList = modelJudgements["judgements"]["judgementList"]
            keep = []
#             judgementList = Judgements(qid, modelJudgements["judgements"]["judgementList"])
            for page in range(1, 3):
                try:
                    searchResults = onsScraper.searchResultsList(searchTerm, method=search_utils.sltr, 
                                            model=model, size=100, page=page, verbose=False)
    
                    searchResultsDecoded = []
                    for searchResult in searchResults:
                        token = searchResult.replace("/redir/", "")
                        uri = getUriFromJwtToken(token)
                        searchResultsDecoded.append(uri)

                    for judgement in judgementList:
                        judgement["rank"] = -1
                        url = judgement["attrs"]["uri"]
                        if (url in searchResultsDecoded):
                            newRank = int(((page - 1) * 10.0) + searchResultsDecoded.index(url) + 1)
                            judgement["rank"] = newRank
                            j = copy.deepcopy(judgement)
                            j["rank"] = newRank
                            keep.append(j)
                    
                except HTTPError:
                    break
                    
            if (len(keep) > 1):
                judgementList = Judgements(qid, keep)
                ndcg = judgementList.ndcg()
                print model, ndcg.mean()
                modelPerformanceDict[model].append(ndcg.mean())
            else:
                modelPerformanceDict[model].append(0)    

qid = 0
# processSearchTerm("cpi", modelPerformanceDict)
for searchTerm in searchTerms:
    qid += 1
    processSearchTerm(searchTerm, modelPerformanceDict)
         

***********************************************
rpi
***********************************************


/Users/sullid/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /Users/sullid/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


ons_model_1 0.0973990058772
ons_model_2 0.0855285161896
ons_model_3 0.153006147017
ons_model_4 0.0462567442842
ons_model_5 0.0973947762352
ons_model_6 0.0281492543824
ons_model_7 0.10079197299
ons_model_8 0.100239100695
ons_model_9 0.0458561550883
all 0.0598182098988
***********************************************
gender pay gap
***********************************************
ons_model_1 0.931990007812
ons_model_2 0.931801430125
ons_model_3 0.939906331244
ons_model_4 0.938087243375
ons_model_5 0.931882820202
ons_model_6 0.0178583225143
ons_model_7 0.931990007812
ons_model_8 0.931990007812
ons_model_9 0.93838265731
all 0.93838265731
***********************************************
cpi
***********************************************
ons_model_1 0.0798315374341
ons_model_2 0.0261645493043
ons_model_3 1.0
ons_model_5 0.0771149804411
ons_model_6 0.0189585071805
ons_model_7 0.0825607748164
ons_model_8 0.063223272839
ons_model_9 0.9
all 0.9
***********************************************
gdp
*

KeyboardInterrupt: 

In [41]:
def printStats(modelPerformanceDict):
    for model in models:
        entry = modelPerformanceDict[model]
        meanNdcg = float(sum(entry)) / float(len(entry))
        print "model: %s, mean ndcg: %f" % (model, meanNdcg)

printStats(modelPerformanceDict)
print "Mean for 'all' was 0.626198"

model: ons_model_1, mean ndcg: 0.343319
model: ons_model_2, mean ndcg: 0.360432
model: ons_model_3, mean ndcg: 0.805518
model: ons_model_4, mean ndcg: 0.104628
model: ons_model_5, mean ndcg: 0.352101
model: ons_model_6, mean ndcg: 0.093701
model: ons_model_7, mean ndcg: 0.355125
model: ons_model_8, mean ndcg: 0.355389
model: ons_model_9, mean ndcg: 0.587732
model: all, mean ndcg: 0.626198
